# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures on the test data you have created.

YOU ARE NOT ALLOWED TO USE ANY OTHER 3RD PARTY LIBRARIES LIKE PANDAS. PLEASE ONLY MODIFY CONTENT INSIDE THE FUNCTION SKELETONS
Please read why: https://www.coursera.org/learn/exploring-visualizing-iot-data/discussions/weeks/3/threads/skjCbNgeEeapeQ5W6suLkA
. Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.
Please also make sure to only implement the function bodies and DON'T add any additional code outside functions since this might confuse the autograder.

So the function below is used to make it easy for you to create a data frame from a cloudant data frame using the so called "DataSource" which is some sort of a plugin which allows ApacheSpark to use different data sources.


In [597]:
#Please don't modify this function
def readDataFrameFromCloudant(host,user,pw,database):
    cloudantdata=spark.read.format("com.cloudant.spark"). \
    option("cloudant.host",host). \
    option("cloudant.username", user). \
    option("cloudant.password", pw). \
    load(database)

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. 

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. You can use this skeleton for all subsequent functions. Everything can be implemented using SQL only if you like.

In [598]:
def minTemperature(df,spark):
    # return spark.sql("SELECT * as mintemp from washing").first().mintemp
    
    # result = spark.sql("SELECT d.temperature from washing WHERE d.temperature IS NOT NULL")
    # return result.show()
    
    # df = df.schema
    # return df
    
    return spark.sql("SELECT MIN(d.temperature) as minTemp from washing WHERE d.temperature IS NOT NULL").first().minTemp

Please now do the same for the mean of the temperature

In [599]:
def meanTemperature(df,spark):
    
    # spark.sql("SELECT SUM(d.temperature) as totalTemp from washing").first().totalTemp
    # mean = df.sum("d.temperature")/float(df.count("d.temperature"))
    # return mean.show()
    # return ##INSERT YOUR CODE HERE##
    
    # solucao do exercicio feito por sql
    sumTemperature = spark.sql("SELECT SUM(d.temperature) as sumTemp from washing WHERE d.temperature IS NOT NULL").first().sumTemp
    numTemperature = spark.sql("SELECT d.temperature as numTemp from washing WHERE d.temperature IS NOT NULL").count()
    mean = sumTemperature/float(numTemperature)
    # ##################################
   
    # solucao do exercicio feito em spark
    # myList = df.select("d.temperature").rdd.flatMap(lambda x: x).collect() # transforma uma column do dataset para uma simples lista 
    # myList = filter(None, myList)
    # rdd = sc.parallelize(myList)
    # mean = rdd.sum()/float(rdd.count())
    # #####################################
    
    return mean

Please now do the same for the maximum of the temperature

In [600]:
def maxTemperature(df,spark):
    
    return spark.sql("SELECT MAX(d.temperature) as maxtemp from washing WHERE d.temperature IS NOT NULL").first().maxtemp

Please now do the same for the standard deviation of the temperature

In [601]:
def sdTemperature(df,spark):
    from math import sqrt
    from pyspark import SparkContext
    # sc=SparkContext()
    sc = SparkContext.getOrCreate()
    
    # solucao do exercicio feito em spark 
    myList = df.select("d.temperature").rdd.flatMap(lambda x: x).collect() # transforma uma column do dataset para uma simples lista 
    myList = filter(None, myList)
    rdd = sc.parallelize(myList)
    n = rdd.count()

    # solucao do exercicio feito por sql
    sumTemperature = spark.sql("SELECT SUM(d.temperature) as sumTemp from washing WHERE d.temperature IS NOT NULL").first().sumTemp
    numTemperature = spark.sql("SELECT d.temperature as numTemp from washing WHERE d.temperature IS NOT NULL").count()
    mean = sumTemperature/float(numTemperature)
    # ##################################
    
    sd = sqrt(rdd.map(lambda x : pow(x - mean,2)).sum()/n)
    # ######################################################
    
    # solucao do exercicio feito em spark 
    # df = spark.sql("SELECT d.temperature as numTemp from washing WHERE d.temperature IS NOT NULL")
    # n = df.count()
    
    return sd




Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four position in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [602]:
def skewTemperature(df,spark):    
# ###### Original Code to Work with SQL Statement, but I prefer to work with RDD
#    return spark.sql("""
# SELECT 
#    (
#        1/##INSERT YOUR CODE HERE##
#    ) *
#    SUM (
#        POWER(##INSERT YOUR CODE HERE##-%s,3)/POWER(%s,3)
#    )
#
# as ##INSERT YOUR CODE HERE## from washing
#                    """ %(meanTemperature(df,spark),sdTemperature(df,spark)))##INSERT YOUR CODE HERE##
# #############################
    from math import sqrt
    from pyspark import SparkContext
    # sc=SparkContext()
    sc = SparkContext.getOrCreate()
    
    myList = df.select("d.temperature").rdd.flatMap(lambda x: x).collect() # transforma uma column do dataset para uma simples lista 
    myList = filter(None, myList)
    rdd = sc.parallelize(myList)
    
    # solucao do exercicio feito por sql
    sumTemperature = spark.sql("SELECT SUM(d.temperature) as sumTemp from washing WHERE d.temperature IS NOT NULL").first().sumTemp
    numTemperature = spark.sql("SELECT d.temperature as numTemp from washing WHERE d.temperature IS NOT NULL").count()
    mean = sumTemperature/float(numTemperature)
    # ##################################
    
    # solucao do exercicio feito em spark 
    n = rdd.count()
    sd = sqrt(rdd.map(lambda x : pow(x - mean,2)).sum()/n)
    
    n = float(n)

    sk = n/((n-1)*(n-2))*rdd.map(lambda x : pow(x - mean,3)/pow(sd,3)).sum()

    return sk


Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [603]:
def kurtosisTemperature(df,spark):   
    from math import sqrt
    from pyspark import SparkContext
    # sc=SparkContext()
    sc = SparkContext.getOrCreate()
    
    myList = df.select("d.temperature").rdd.flatMap(lambda x: x).collect() # transforma uma column do dataset para uma simples lista 
    myList = filter(None, myList)
    rdd = sc.parallelize(myList)
    
    # solucao do exercicio feito por sql
    sumTemperature = spark.sql("SELECT SUM(d.temperature) as sumTemp from washing WHERE d.temperature IS NOT NULL").first().sumTemp
    numTemperature = spark.sql("SELECT d.temperature as numTemp from washing WHERE d.temperature IS NOT NULL").count()
    mean = sumTemperature/float(numTemperature)
    # ##################################
    
    # solucao do exercicio feito em spark 
    n = rdd.count()
    sd = sqrt(rdd.map(lambda x : pow(x - mean,2)).sum()/n)
    
    n = float(n)
    kurtosis = rdd.map(lambda x : pow(x - mean,4)/pow(sd,4)).sum()/(1-n)
    
    return kurtosis

Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [604]:
def correlationTemperatureHardness(df,spark):
    from math import sqrt
    from pyspark import SparkContext
    # sc=SparkContext()
    sc = SparkContext.getOrCreate()
    
    # result = spark.sql("SELECT d.* from washing")
    # return result.show()

     # fazendo o calculo do mean e do sd para temperatura
    sumTemperature = spark.sql("SELECT SUM(d.temperature) as sumTemp from washing WHERE d.temperature IS NOT NULL").first().sumTemp
    numTemperature = spark.sql("SELECT d.temperature as numTemp from washing WHERE d.temperature IS NOT NULL").count()
    meanTemp = sumTemperature/float(numTemperature)
    # ##################################
    
    # fazendo o calculo do mean e do sd para hardness
    sumHard = spark.sql("SELECT SUM(d.hardness) as sumHard from washing WHERE d.hardness IS NOT NULL").first().sumHard
    numHard = spark.sql("SELECT d.hardness as numHard from washing WHERE d.hardness IS NOT NULL").count()
    meanHard = sumHard/float(numHard)
    # ##################################                         
                             
       
    # #################################
    myListTemp = df.select("d.temperature").rdd.flatMap(lambda x: x).collect() # transforma uma column do dataset para uma simples lista 
    myListTemp = filter(None, myListTemp)
    rddTemp = sc.parallelize(myListTemp)
    
    myListHard = df.select("d.hardness").rdd.flatMap(lambda x: x).collect() # transforma uma column do dataset para uma simples lista 
    myListHard = filter(None, myListHard)
    rddHard = sc.parallelize(myListHard)
    # #################################
    
    rddTH = rddTemp.zip(rddHard)
    
    # #################################
    
    covTH = rddTH.map(lambda (x,y) : (x - meanTemp)*(y - meanHard)).sum()/rddTH.count()

    # ######################################################
        
    n = rddTH.count()
    
    # ######################################################
    
    sdTemp = sqrt(rddTemp.map(lambda x : pow(x - meanTemp,2)).sum()/n)
    sdHard = sqrt(rddHard.map(lambda x : pow(x - meanHard,2)).sum()/n)
    
    # ######################################################
    
    # calculo de ambos os dados, temperature e hardness

    corrTH = covTH/(sdTemp * sdHard)
    
    return corrTH  

### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED
#axx
### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED

In [605]:
#TODO Please provide your Cloudant credentials here
hostname = "0f8410a3-0e0f-467f-a3cf-b9fbd5da37c2-bluemix.cloudant.com"
user = "0f8410a3-0e0f-467f-a3cf-b9fbd5da37c2-bluemix"
pw = "1363e31faf953220b28f78a9afae906603f2882db47b0e9375bf650569f4d368"
database = "washing"
cloudantdata=readDataFrameFromCloudant(hostname, user, pw, database)

+--------------------+--------------------+--------------------+
|                 _id|                _rev|                   d|
+--------------------+--------------------+--------------------+
|0f83931363c7ce12d...|1-ffedf0982164b41...|[2,null,null,74,n...|
|0f83931363c7ce12d...|1-25fb1d3ec6eecb7...|[2,null,null,null...|
|0f83931363c7ce12d...|1-69c8b2d638db12e...|[10,11,acceptable...|
|0f83931363c7ce12d...|1-2e2a41b45fcb93a...|[11,11,acceptable...|
|0f83931363c7ce12d...|1-7f45175964250f8...|[5,null,null,68,n...|
|0f83931363c7ce12d...|1-75f7f3cdb9a3787...|[17,11,acceptable...|
|0f83931363c7ce12d...|1-a53fb4cd7965f0f...|[20,11,acceptable...|
|0f83931363c7ce12d...|1-2507d6eb21e3936...|[34,11,acceptable...|
|0f83931363c7ce12d...|1-fc51c2dbb04a718...|[43,11,acceptable...|
|0f83931363c7ce12d...|1-91dd8f0ce02523b...|[50,11,acceptable...|
|0f83931363c7ce12d...|1-3685ecced4c3b9f...|[51,11,acceptable...|
|0f83931363c7ce12d...|1-71d10c74297acf1...|[55,11,acceptable...|
|0f83931363c7ce12d...|1-c

In [606]:
minTemperature(cloudantdata,spark)

80

In [607]:
meanTemperature(cloudantdata,spark)

89.92349726775956

In [608]:
maxTemperature(cloudantdata,spark)

100

In [609]:
sdTemperature(cloudantdata,spark)

6.088567160194603

In [610]:
skewTemperature(cloudantdata,spark)

0.04162061140856722

In [611]:
kurtosisTemperature(cloudantdata,spark)

-1.7171003066322552

In [612]:
correlationTemperatureHardness(cloudantdata,spark)

-0.07156349787356674

Congratulations, you are done, please download this notebook as python file using the export function and submit is to the gader using the filename "assignment3.1.py"